<a href="https://colab.research.google.com/github/alendoko/bina/blob/main/LB4/DATA/Python_JUPYTER_Data_Analysis_SteyByStep_Melbourne_Houseprice2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jupiter Notebook, welches sich die Daten von bestehenden PV-Anlagen in der Schweiz analysiert

## Daten laden, aufbereiten und anreichern

In [ ]:
import pandas as pd

# Load CSV File
file_path = '../data/electricity_production_plants/ElectricityProductionPlant.csv'  # Replace with the path to your CSV file
df = pd.read_csv(file_path)  # Read the CSV file into a DataFrame, handling bad lines and ensuring comma as separator


main_categories_file_path = '../data/electricity_production_plants/MainCategoryCatalogue.csv'  # Path to the replacement file
main_categories = pd.read_csv(main_categories_file_path)
main_categories_dict = dict(zip(main_categories['Catalogue_id'], main_categories['de']))
df['MainCategory'] = df['MainCategory'].replace(main_categories_dict)

sub_categories_file_path = '../data/electricity_production_plants/SubCategoryCatalogue.csv'  # Path to the replacement file
sub_categories = pd.read_csv(sub_categories_file_path)
sub_categories_dict = dict(zip(sub_categories['Catalogue_id'], sub_categories['de']))
df['SubCategory'] = df['SubCategory'].replace(sub_categories_dict)

plant_categories_file_path = '../data/electricity_production_plants/PlantCategoryCatalogue.csv'  # Path to the replacement file
plant_categories = pd.read_csv(plant_categories_file_path)
plant_categories_dict = dict(zip(plant_categories['Catalogue_id'], plant_categories['de']))
df['PlantCategory'] = df['PlantCategory'].replace(plant_categories_dict)

# Display the first few rows of the DataFrame to verify the data
df.head()


"""
    TODO Visualisierungen
     - Anzahl der potenziellen Gesamtleistung für alle Kantone aufführen
     - Entwicklung der Gesamtleistung je Kanton aggregiert mit Regler für Zeitraum 2015-2024 erstellen. Wie sind die Unterschiede, gibt es Auffälligkeiten?
     - Karte mit allen Kraftwerken und deren Leistung erstellen
"""

## Karte mit allen PV-Anlagen in der Schweiz, Stand 2024

In [ ]:
import folium
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import display
from pyproj import Transformer
from folium.plugins import FastMarkerCluster

# Filter the dataset for Photovoltaik Anlagen
df_photovoltaik = df[df['SubCategory'] == 'Photovoltaik']

# Convert the 'BeginningOfOperation' column to datetime
df_photovoltaik = df_photovoltaik.copy()
df_photovoltaik['BeginningOfOperation'] = pd.to_datetime(df_photovoltaik['BeginningOfOperation'], errors='coerce')

# Filter the data for the years to 2024
df_photovoltaik = df_photovoltaik[df_photovoltaik['BeginningOfOperation'].dt.year <= 2024]

# Create a folium map centered on Switzerland
map = folium.Map(location=[46.8, 8.33], zoom_start=7, tiles='TopPlusOpen.Color', attr='Map data: &copy; <a href="http://www.govdata.de/dl-de/by-2-0">dl-de/by-2-0</a>')

# Add a FastMarkerCluster for better performance with large datasets
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)
markerCluster = FastMarkerCluster(
    [[transformer.transform(row['_x'], row['_y'])[1], transformer.transform(row['_x'], row['_y'])[0]]
     for _, row in df_photovoltaik.iterrows() if not pd.isna(row['_x']) and not pd.isna(row['_y'])]
).add_to(map)

# HTML-Code für den Titel
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>PV-Anlagen in der Schweiz, Stand 2024</b></div>
'''
# Hinzufügen des Titels zur Karte
map.get_root().html.add_child(folium.Element(title_html))

# Speichern der Karte in einer HTML-Datei
map.save('../docs/assets/diagramme/elcoms_map.html')

# Anzeigen der Karte
map

